# Discrete State Seeker
Consider a robot with state variable $x = (x_0,x_1,...)$ where $x_t \in \mathcal{X}$ for every step $t \in \mathbb{N}$. At every step $t$, the robot can choose an action $a_t \in \mathcal{A}$ according to policy $a_t^\pi=\pi(x_t)$ that stochastically sets the robot's state to $x_{t+1}=Δ(x_t,a)$ and earns some reward $r(x_t,x_{t+1},a)$. The robot's goal is to maximize the total reward over an infinite time horizon with discount factor $\rho$: $\pi^\star = \argmax_{\pi}{\sum_{t=0}^{\infty} {\rho^t r(x_t, x_{t+1}, a_t^\pi)}}$.

We will use the off-policy TD algorithm (Q-learning) to estimate $Q(x,a)$, the expected reward remaining when taking action $a$ at state $x$; the $\pi^\star$ will then simply be to choose the action $a$ that maximizes $Q(x,a)$ at any state $x$. We will achieve this through:
1. Let $\alpha = (\alpha_0, \alpha_1, ...)$ be the learning rate with $\alpha_n \in (0,1)$, $\sum_n \alpha_n = \infty$, $\sum_n \alpha_n^2 < \infty$
2. Randomly generate an initial $Q_0$ matrix and an initial $x_0$. Choose some randomized policy $\pi$.
3. For $n=0,1,2,...$
    - Suppose $Q_n$ and $x_n$ are known. Choose action $a_n$ according to $\pi$ and $x_{n+1} = Δ(x_n,a_n)$.
    - Calculate updated values for $Q_{n+1}(x_n,a_n)$ based on the following equation:
        $$Q_{n+1}(x_n,a_n) = Q_n(x_n,a_n) + \alpha_n (r(x_n, x_{n+1}, a_n) + \rho \sup_{b \in \mathcal{A}} Q_n(x_{n+1},b) - Q_n(x_n,a_n))$$
4. Repeat the recursion until $Q_{n+1} \approx Q_n$.

In [13]:
import numpy as np

In [14]:
class Robot:
    def __init__(self, state_space, action_space, discount, next_state, reward):
        self.state_space = state_space # \mathcal{X}  = list of possible states
        self.action_space = action_space # \mathcal{A} = list of possible actions at any state
        self.discount = discount # \rho = the discount factor
        self.next_state = next_state # Δ(x_n,a) = the callable that stochastically determines x_{n+1}
        self.reward = reward # r(x_n, x_{n+1}, a) = the reward for going from state x_n to x_{n+1} through action a

        # other instance variables
        self.Q = np.zeros(shape=(state_space.size, action_space.size))

    # resets the Q matrix to be all zeros
    def reset_Q(self):
        self.Q = np.zeros(shape=(self.state_space.size, self.action_space.size))

    # runs the Q-learning algorithm
    # we use x_0 = 0 and use a uniformly random policy for training
    def train_Q(self, learning_rate, max_iterations, tolerance):
        x = 0
        for step in range(max_iterations):
            # determine a and x_{n+1}
            a = np.random.choice(self.action_space)
            x_next = self.next_state(x, a)

            # find Q_{n+1}
            Q_update = self.Q[x, a] + learning_rate(step) * (
                self.reward(x, x_next, a) +
                self.discount * np.max(self.Q[x_next,]) -
                self.Q[x,a]
            )

            # check recursion end condition
            if (Q_update - self.Q[x,a] < tolerance):
                return

            # set Q = Q_{n+1}
            self.Q[x, a] = Q_update

    # returns the best action to take at state x
    def best_action(self, x):
        return np.argmax(self.Q[x,])

In [20]:
p_h = 0.7 # probability of remaining high battery upon searching
p_l = 0.6 # probability of remaining low battery upon searching
# (x, x_next, action) -> probability
prob_matrix = np.array([
        [
            [p_l,    1.0,    0.0], # low to low
            [1-p_l,  0.0,    1.0], # low to high
        ], [
            [1-p_h,  0.0,    0.0], # high to low
            [p_h,    1.0,    1.0], # high to high
        ],
    ])

r_s = 1 # reward for searching
r_w = 0.1 # reward for waiting
r_p = -10 # reward for running out of battery (low -> high)
# (x, x_next, action) -> reward
reward_matrix = np.array([
        [
            [r_s,   r_w,     np.nan], # low to low
            [r_p,   np.nan,  0], # low to high
        ], [
            [r_s,   np.nan,  np.nan], # high to low
            [r_s,   r_w,     0], # high to high
        ],
    ])

recycler = Robot(
    state_space = np.array([0, 1]), # low, high
    action_space = np.array([0, 1, 2]), # search, wait, charge
    discount = 0.9,
    next_state = lambda x,a : 0 if np.random.rand() < prob_matrix[x,:,a][0] else 1,
    reward = lambda x,x_next,a : reward_matrix[x,x_next,a],
)

In [22]:
recycler.reset_Q()
recycler.train_Q()

TypeError: Robot.train_Q() missing 3 required positional arguments: 'learning_rate', 'max_iterations', and 'tolerance'